In [1]:
from langchain_ollama import ChatOllama, OllamaEmbeddings

llm = ChatOllama(model="llama3.1", temperature=0)
ollama_nomic_embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [ ]:
from langchain_community.document_loaders.pdf import PyPDFLoader
# Initializing the PyPDFLoader with the file path of the PDF document you want to load
# In this case, the file "FaustinaLazarusCV.pdf" is provided as the path to the PDF document
loader = PyPDFLoader(file_path="C:\Users\manoj singh\Desktop\RESUME (1).pdf")
data = loader.load()

SyntaxError: invalid syntax. Perhaps you forgot a comma? (217856585.py, line 4)

In [ ]:
# Importing the RecursiveCharacterTextSplitter from langchain_text_splitters for splitting long text into smaller chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 'chunk_size' defines the maximum length of each chunk (in this case, 1024 characters)
# 'chunk_overlap' defines how much overlap should be there between adjacent chunks (in this case, 200 characters)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=200)

# Splitting the document data (extracted PDF content) into smaller chunks using the 'split_documents' method
splits = text_splitter.split_documents(data)

In [ ]:
from langchain_chroma import Chroma

# Defining a batch size limit for processing the splits
batch_size_limit = 4096  # Maximum number of characters allowed per batch

# Initializing variables to track batch processing
current_batch_size = 0  # Tracks the size of the current batch
batch_splits = []  # Holds the current batch of document splits
all_splits = []  # Holds all the document splits processed so far

# Setting the collection name and persist directory for the vector store
collection_name = "faustina_lazarus_cv"  # Name for the vector store collection
persist_directory = f"./{collection_name}"  # Directory where the vector store data will be persisted

# Initializing the Chroma vector store, which will store the document vectors and support retrieval
# The vector store uses Ollama embeddings (from the 'ollama_nomic_embeddings' object) to create document embeddings
vectorstore = Chroma(collection_name=collection_name, persist_directory=persist_directory, embedding_function=ollama_nomic_embeddings)

# Looping through each split from the previous step (text chunks from the PDF)
for split in splits:
    split_size = len(split.page_content)  # Get the size of the current document chunk
    # Check if adding the current chunk exceeds the batch size limit
    if current_batch_size + split_size > batch_size_limit:
        # If so, add the current batch of splits to the vector store
        vectorstore.add_documents(documents=batch_splits)
        all_splits.extend(batch_splits)  # Add the current batch to the all_splits list
        batch_splits = [split]  # Start a new batch with the current split
        current_batch_size = split_size  # Reset the batch size to the current split size
    else:
        # If the batch size doesn't exceed the limit, add the split to the current batch
        batch_splits.append(split)
        current_batch_size += split_size  # Increase the batch size by the size of the current split

# After processing all splits, if there are any remaining splits in the batch, add them to the vector store
if batch_splits:
    vectorstore.add_documents(documents=batch_splits)
    all_splits.extend(batch_splits)

# Reinitialize the Chroma vector store and set it up as a retriever to handle search queries or document retrieval
vectorstore = Chroma(collection_name=collection_name, persist_directory=persist_directory, embedding_function=ollama_nomic_embeddings)
retriever = vectorstore.as_retriever()  # Convert the vector store into a retriever for querying


In [ ]:
# Defining a system prompt template for generating responses based on a given context
system_prompt = (
    """
    Answer the questions based on the provided context only.
    Please provide the most accurate response based on the question
    <context>
    {context}
    <context>
    Questions:{input}
    """
)

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate

# Defining the system prompt that instructs the model to reformulate a user question based on chat history
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

# - The system message uses the 'contextualize_q_system_prompt' to guide the model in reformulating the question
# - The 'chat_history' placeholder will dynamically insert the previous conversation history
# - The 'human' message inserts the user's latest input (question) as a dynamic field
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
# This setup helps retrieve documents or responses while considering the context from prior interactions in the chat
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Defining the question-answering prompt template for the model
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),  # The system message that provides instructions on how to answer the question
        MessagesPlaceholder("chat_history"),  # A placeholder for the chat history to provide context to the model
        ("human", "{input}"),  # A placeholder for the user's question (input)
    ]
)

# Creating a "stuff" document chain, which combines document retrieval and processing using the 'qa_prompt' template
# The 'stuff' document chain will guide the model in answering questions based on relevant documents and context
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Creating a retrieval chain that uses the history-aware retriever and the question-answer chain
# This chain allows the system to retrieve contextually relevant documents and then generate an answer using the 'question_answer_chain'
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

while True:
    question = input("Enter your question...")

    if question in ["quit", "exit"]:
        break

    ai_msg = rag_chain.invoke({"input": question, "chat_history": chat_history})
    print(f"Question: {question}")
    print(f"Answer: {ai_msg['answer']}")
    print("**********************************************************************")

    chat_history.extend([
        HumanMessage(content=question),
        AIMessage(content=ai_msg["answer"])
    ])